# 神经网络

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tensorboardX import SummaryWriter
from torchsummary import summary

# 定义训练的设备 使用gpu进行训练
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 输入图像channel：1；输出channel：6；5x5卷积核
        # in  1*  32*32  => 6* 28*28 
        self.conv1 = nn.Conv2d(1, 6, 5)
        # in 6 * 14*14  => 16*   10*10 
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 仿射变换：y = Wx + b ， in : 16*5*5 | 400 = > 120 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        # 2x2 Max pooling
        # in    28*28     => 14*14 
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        # in 10*10   => 5*5 
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]  # 除去批大小维度的其余维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()
model = net.to(device)

print(net)

summary(model, (1, 32, 32))


writer = SummaryWriter("torchlogs/") 
dummy_input01 = torch.rand(10, 1, 32, 32)  # 假设输入10张1*28*28的图片
dummy_input01 = dummy_input01.to(device)
writer.add_graph(model, dummy_input01)
writer.close()

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
            Conv2d-2           [-1, 16, 10, 10]           2,416
            Linear-3                  [-1, 120]          48,120
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.24
Estimated Total Size (MB

In [11]:
params = list(net.parameters())
print(len(params))
for param in params:
    print(param .size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [13]:
input_cpu = torch.rand(10, 1, 32, 32)  # 假设输入10张1*28*28的图片
input = input_cpu.to(device)

out = net(input)
print(out)

tensor([[ 0.0446, -0.1291,  0.0909,  0.0149,  0.0414, -0.0109, -0.0063, -0.0150,
         -0.0986,  0.0650],
        [ 0.0442, -0.1281,  0.0923,  0.0166,  0.0400, -0.0149, -0.0061, -0.0120,
         -0.1014,  0.0664],
        [ 0.0445, -0.1302,  0.0893,  0.0159,  0.0410, -0.0135, -0.0047, -0.0141,
         -0.1006,  0.0640],
        [ 0.0414, -0.1303,  0.0933,  0.0140,  0.0411, -0.0142, -0.0063, -0.0130,
         -0.1028,  0.0654],
        [ 0.0475, -0.1274,  0.0920,  0.0152,  0.0408, -0.0146, -0.0074, -0.0116,
         -0.1046,  0.0620],
        [ 0.0411, -0.1293,  0.0922,  0.0164,  0.0395, -0.0172, -0.0094, -0.0107,
         -0.0976,  0.0657],
        [ 0.0471, -0.1288,  0.0907,  0.0167,  0.0368, -0.0134, -0.0058, -0.0110,
         -0.1005,  0.0649],
        [ 0.0437, -0.1290,  0.0918,  0.0170,  0.0400, -0.0134, -0.0088, -0.0137,
         -0.1029,  0.0648],
        [ 0.0441, -0.1327,  0.0907,  0.0168,  0.0392, -0.0113, -0.0098, -0.0148,
         -0.1004,  0.0647],
        [ 0.0444, -